In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-10-03 21:01:12--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.22MB/s    in 0.8s    

2021-10-03 21:01:14 (1.22 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R8MEA6IGAHO0B|          5|            0|          0|   N|                Y|
|R31LOQ8JGLPRLK|          5|            0|          1|   N|                Y|
|R2Y0MM9YE6OP3P|          5|            0|          0|   N|                Y|
| RRB9C05HDOD4O|          4|            0|          0|   N|                Y|
|R26I2RI1GFV8QG|          2|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [36]:
#Make df where total votes are greater or equal to 20
high_vote_count_df = vine_df.filter("total_votes>=20")
high_vote_count_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RBEMQ29WJBHYG|          5|          164|        168|   N|                Y|
|R2JK5Y8D5MXAGP|          1|            5|        121|   N|                Y|
|R13W3EMIUV120L|          4|          288|        294|   N|                Y|
|R30TJ8POYNVCRE|          5|           15|         21|   N|                N|
|R1H5Y9Z2DHRNVO|          1|            4|         95|   N|                Y|
|R1LK03P7BCJEJD|          1|           31|         35|   N|                Y|
| RBXUICKGHL685|          1|           48|         52|   N|                N|
|R2E16WLZKP78GO|          5|           19|         21|   N|                Y|
|R1EN4PGQIFSENW|          5|           27|         28|   N|                Y|
| RXZWQU66AHIYG|          5|           24|         25|   N|     

In [37]:
# Make a Df where the columns for helpful_votes are divided by total_votes, and only keep the rows whose percentage of this is greater than 50%
import pyspark.sql.functions as F
helpful_ratio_df = high_vote_count_df.withColumn("helpful_percent", (F.col("helpful_votes"))/(F.col("total_votes")))
majority_helpful_df = helpful_ratio_df.filter("helpful_percent>=.5")
percentage_maj_helpful_df = majority_helpful_df.withColumn("helpful_percent",(F.col("helpful_percent")*100))
percentage_maj_helpful_df.show(10)


+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  helpful_percent|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
| RBEMQ29WJBHYG|          5|          164|        168|   N|                Y|97.61904761904762|
|R13W3EMIUV120L|          4|          288|        294|   N|                Y|97.95918367346938|
|R30TJ8POYNVCRE|          5|           15|         21|   N|                N|71.42857142857143|
|R1LK03P7BCJEJD|          1|           31|         35|   N|                Y|88.57142857142857|
| RBXUICKGHL685|          1|           48|         52|   N|                N| 92.3076923076923|
|R2E16WLZKP78GO|          5|           19|         21|   N|                Y|90.47619047619048|
|R1EN4PGQIFSENW|          5|           27|         28|   N|                Y|96.42857142857143|
| RXZWQU66AHIYG|          5|           2

In [41]:
# Make a DF for only the vine program reviewers
in_vine_prog_df = percentage_maj_helpful_df.filter(percentage_maj_helpful_df.vine.contains('Y'))
in_vine_prog_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  helpful_percent|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R3BOQTLUU3Y4L8|          4|           42|         55|   Y|                N|76.36363636363637|
| RWF03LXVXC22A|          3|          396|        445|   Y|                N|88.98876404494382|
| REPU28WG1VZUE|          5|          242|        281|   Y|                N|86.12099644128114|
|R1D6REC9HPJVQY|          4|           31|         41|   Y|                N|75.60975609756098|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+



In [40]:
# Make a df for the non-vine program reviewers
not_vine_prog_df = percentage_maj_helpful_df.filter(percentage_maj_helpful_df.vine.contains('N'))
not_vine_prog_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  helpful_percent|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
| RBEMQ29WJBHYG|          5|          164|        168|   N|                Y|97.61904761904762|
|R13W3EMIUV120L|          4|          288|        294|   N|                Y|97.95918367346938|
|R30TJ8POYNVCRE|          5|           15|         21|   N|                N|71.42857142857143|
|R1LK03P7BCJEJD|          1|           31|         35|   N|                Y|88.57142857142857|
| RBXUICKGHL685|          1|           48|         52|   N|                N| 92.3076923076923|
|R2E16WLZKP78GO|          5|           19|         21|   N|                Y|90.47619047619048|
|R1EN4PGQIFSENW|          5|           27|         28|   N|                Y|96.42857142857143|
| RXZWQU66AHIYG|          5|           2

In [49]:
#Count Number of Entries that were in the vine program
vine_count = in_vine_prog_df.count()
vine_count

4

In [50]:
#Count Number of Entries Not in Vine Program
not_vine_count = not_vine_prog_df.count()
not_vine_count

1064

In [53]:
# Count the 5 star ratings by vine program reviewers (this could be done by hand on such a small df, but done in code for class purposes)
five_star_vine = in_vine_prog_df.filter("star_rating=5").count()
five_star_vine

1

In [52]:
# Count the 5 star ratings by non-vine program reviewers
five_star_not_vine = not_vine_prog_df.filter("star_rating=5").count()
five_star_not_vine

527

In [56]:
# Percentage of 5 star ratings by Vine Program reviewers
vine_fivestar_pct = (five_star_vine/vine_count)*100
vine_fivestar_pct

25.0

In [57]:
# Percentage of 5 star ratings by Non-Vine Program reviewers
not_vine_fivestar_pct = (five_star_not_vine/not_vine_count)*100
not_vine_fivestar_pct

49.53007518796993